### Linear Time Selection

In this notebook we will look at two types of selection algorithms, the RSelect, or the randomized selection algorithm and a deterministic version called DSelect. But before we proceed the question is "What is selection?" A selection problem aims to find the $i^{th}$ smallest number for n given numbers. The simplest approach would be to sort the input array and then select the $i^{th}$ position number essentially doing the selection in $nlogn$ time. This however is not the optimum way as selection if an easier problem than sorting and thus we should achieve this in much less running time than sorting. To select a number from n given numbers, we will have to view all the numbers at the minimum and thus $O(n)$ is a reasonable lower bound and we know that sorting can be done in $O(nlogn)$ which is an upperbound for our problem.

Can we do something to possibly perform the selection in $O(n)$ time? Let us implement the selection problem using the partition routine we used in quick sort.



In [3]:
def swap(in_array, ix1, ix2):
    t = in_array[ix1]
    in_array[ix1] = in_array[ix2]
    in_array[ix2] = t
    
def partition(in_array, start_ix, end_ix):

    partition_idx = start_ix
    i = start_ix
    pivot = in_array[partition_idx]
    for idx in range(start_ix + 1, end_ix):
        if in_array[idx] <=  pivot:
            i += 1
            if idx != i:
                swap(in_array, i, idx)

    swap(in_array, start_ix, i)
    partition_idx = i
    
    return partition_idx

def RSelect(array, i, start_ix = 0, end_ix = -1):
    #From a given array, find the ith order statistic
    end_ix = len(array) if end_ix == -1 else end_ix    
    pivot_idx  = partition(array, start_ix, end_ix)    
    stat = pivot_idx - start_ix + 1
    if i == stat:
        return array[pivot_idx]
    elif i < stat:
        return RSelect(array, i, start_ix, pivot_idx)
    else:
        return RSelect(array, i - stat, pivot_idx + 1, end_ix)

In [4]:
print('1st Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 1))
print('2nd Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 2))
print('3rd Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 3))
print('4th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 4))
print('5th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 5))
print('6th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 6))
print('7th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 7))
print('8th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 8))

1st Order stat is 1
2nd Order stat is 2
3rd Order stat is 3
4th Order stat is 4
5th Order stat is 5
6th Order stat is 6
7th Order stat is 7
8th Order stat is 8


We will test the implementation with some of the test data given [here](http://theory.stanford.edu/~tim/algorithmsilluminated.html)

In [13]:
with open('problem6.5test1.txt', 'r') as f:
    linest1 = [int(line.strip()) for line in f.readlines()]

with open('problem6.5test2.txt', 'r') as f:
    linest2 = [int(line.strip()) for line in f.readlines()]

print('5th Order statistic of the content expected to be 5469, got', RSelect(linest1, 5))
print('50th Order statistic of the content expected to be 4715, got', RSelect(linest2, 50))

5th Order statistic of the content expected to be 5469, got 5469
50th Order statistic of the content expected to be 4715, got 4715




From above tests, the randomized implementation works in place with constant extra memory and works on a similar way quick sort does. The only difference is that after partition we either recurse on the left or the right side of the array unlike sorting where we recurse on both partitions around pivot. We will prove that the above algorithm runs in linear time.

---
**Intutition**

The partition sub routine does exactly what it does in case of quick sort. Once we choose the partition routine, we know in linear time, what position the pivot element will end up in after the recursve calls. Once we know that value, we can then recurse on the left or the right split after the pivot. In the best case suppose we end up picking the median of the input array as pivot, in which case the array will be split in exactly halves. Since we always do linear work outside the recursive calls, we can express our recurrance as follows
$$T(n)\:=\:T(\frac{n}{2}) + O(n) $$

By master method, a = 1, b = 2 and d = 1, we have $a < b^d$ which is case 2 of the master method. The running time is in this case dominated by the work done outside recursive calls giving us the time as $O(n^d)$ = $O(n)$ in this case. This shows us that if we get an approximate median and split the array into two approximately equal pieces, we can expect the selection problem to run in linear time. Let us see how exactly we get linear time on an average for a given array using the following proof.

---
**Proof**

We will track the progress of our selection problem in phases. we will call the problem is in phase j if it is operating on an array of length $(\frac{3}{4})^{j + 1}$ to $(\frac{3}{4})^{j}$. For example, it will be in phase 0, if the size of the array is between 0.75n to n, phase 1 of the size array is 0.56n to 0.75n and so on. The maximum value of phase j when the value of the input size will be 1 should be $log_{4/3}n$.

We will now define a random variable $X_j$ which gives is the number of times the selection process made recursive calls in phase j. The minimum value of this variable is 0.

The maximum size of the array in phase j is $(\frac{3}{4})^j$ and thus the work done in this phase is no more than $c\cdot (\frac{3}{4})^j \cdot n$. Thus

$$Running\:time\:of\:RSelect\:is\:\leq \sum_{j \geq 0} c\cdot (\frac{3}{4})^j\cdot n X_j$$

By linearity of expectations

$$E[Running\:time\:of\:RSelect\:is]\:\leq cn \sum_{j \geq 0} (\frac{3}{4})^jE[X_j]$$

How do we find $E[X_j]$?

Whenever we choose an approximate median as pivot, a number that gives us 25-75% split of the input array, we are guaranteed to procees to next phase. If only when the split produces an array giving is greater than 75% of elements remaining after partition, we stay in the phase j for another recursive call. Thus we can conclude, picking an approximate median is guaranteed to take us to next phase.

Secondly, picking an approximate median has a probability of 50%. Suppose we have n numbers say, 1 to 100, then by picking anything from 26 to 75 is an approximate median which makes up 50% of the input numbers.

The random variable $X_j$ is similar to a coin flipping problem. Suppose N is a random variable which counts the number of times we need to flip the coin to get heads (or tails).

The random variable $X_j$ and $N$ are similar but the $E[X_j] \leq E[N]$

- The coin flopping experiment has to flip the coin atleast once to get heads. The value of X_j can be 0 if we completely skip the phase.
- We have maximum probability of 0.5 of not going out of phase j and staying in the same phase. This is similar to 0.5 probability we get tails and we we need to flip the coin another time.

The expected value of the coin flip experiment is 

$E(N)\: = \: 1\: +\: \frac{1}{2}E(N)$

The only value of E(N) that satisfies the above equation is 2. The equation is explained as follows. The value 1 is for that minimum one flip needed and the value 0.5 is the probability that we get tails. We can thus say that $E(X_j) \leq 2$

Geometric sequence $1\: + \: r^2\: + \: r^3 \dots r^k = \frac{1 - r^{k + 1}}{1 - r}$

This $\sum_{j \geq 0} (\frac{3}{4})^j\: \leq\: \frac{1}{1 - \frac{3}{4}} = 4$

Therefore, 
$E[Running\:time\:of\:RSelect\:is]\:\leq cn \sum_{j \geq 0} (\frac{3}{4})^jE[X_j] = 4\cdot 2cn = 8cn$

Hence, the running time of randomized select algorithm on an average runs in $O(n)$

---

Next we will look at an algorithm which does the selection in linear time and is deterministic. Its called **DSelect** in our notebook

TODO: Give implementation and the mathematical proof of running time.


